# Fake News Detection

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import re
import string

In [2]:
df_fake = pd.read_csv('Fake.csv')
df_true = pd.read_csv('True.csv')

In [3]:
df_fake.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [4]:
#Add encoding column
df_true["class"] = 1
df_fake["class"] = 0

In [5]:
df_true.head()

,title,text,subject,date,class
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",1
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",1
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",1
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017",1
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017",1


In [6]:
df_fake.shape, df_true.shape

((23481, 5), (21417, 5))

In [7]:
#generate test file from the dataset(s)
df_fake_manual_testing = df_fake.tail(10)
for i in range(23480, 23470, -1):
    df_fake.drop([i], axis=0, inplace=True)

df_true_manual_testing = df_true.tail(10)
for i in range(21416, 21406, -1):
    df_true.drop([i], axis=0, inplace=True)

In [8]:
df_manual_testing = pd.concat([df_fake_manual_testing, df_true_manual_testing], axis=0)
df_manual_testing.to_csv("manual_testing.csv")

In [9]:
df_merge = pd.concat([df_fake, df_true], axis=0)
df_merge.head(10)

,title,text,subject,date,class
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",0
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",0
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",0
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",0
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",0
5,Racist Alabama Cops Brutalize Black Boy While...,The number of cases of cops brutalizing and ki...,News,"December 25, 2017",0
6,"Fresh Off The Golf Course, Trump Lashes Out A...",Donald Trump spent a good portion of his day a...,News,"December 23, 2017",0
7,Trump Said Some INSANELY Racist Stuff Inside ...,In the wake of yet another court decision that...,News,"December 23, 2017",0
8,Former CIA Director Slams Trump Over UN Bully...,Many people have raised the alarm regarding th...,News,"December 22, 2017",0
9,WATCH: Brand-New Pro-Trump Ad Features So Muc...,Just when you might have thought we d get a br...,News,"December 21, 2017",0


In [10]:
#remove columns from the df
df = df_merge.drop(["title", "subject", "date"], axis=1)
df.head(10)

,text,class
0,Donald Trump just couldn t wish all Americans ...,0
1,House Intelligence Committee Chairman Devin Nu...,0
2,"On Friday, it was revealed that former Milwauk...",0
3,"On Christmas day, Donald Trump announced that ...",0
4,Pope Francis used his annual Christmas Day mes...,0
5,The number of cases of cops brutalizing and ki...,0
6,Donald Trump spent a good portion of his day a...,0
7,In the wake of yet another court decision that...,0
8,Many people have raised the alarm regarding th...,0
9,Just when you might have thought we d get a br...,0


In [11]:
#shuffling the dataset
df = df.sample(frac=1)

In [12]:
df.head(10)

,text,class
6160,BERLIN (Reuters) - Germany will need a new eco...,1
995,"Not long ago, Rep. Jason Chaffetz (R-Utah), sa...",0
2161,"Trump influencer, conspiracy theorist Alex Jon...",0
6949,WASHINGTON (Reuters) - Congress is expected to...,1
22221,In response to the establishment media s cont...,0
2163,WASHINGTON (Reuters) - Mexico’s Economy Minist...,1
10673,"AUSTIN, Texas (Reuters) - The Texas Court of C...",1
13106,It s funny how things work out the FBI scapego...,0
7667,NEW YORK (Reuters) - With the U.S. election ju...,1
20791,Are we sure this guy is actually qualified for...,0


In [13]:
#checking for null values
df.isnull().sum()

text     0
class    0
dtype: int64

In [14]:
#removing special characters function
def word_drop(text):
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub("\\W", " ",text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

In [15]:
df["text"] = df["text"].apply(word_drop)

In [16]:
df.head(10)

,text,class
6160,berlin reuters germany will need a new eco...,1
995,not long ago rep jason chaffetz r utah sa...,0
2161,trump influencer conspiracy theorist alex jon...,0
6949,washington reuters congress is expected to...,1
22221,in response to the establishment media s cont...,0
2163,washington reuters mexico s economy minist...,1
10673,austin texas reuters the texas court of c...,1
13106,it s funny how things work out the fbi scapego...,0
7667,new york reuters with the u s election ju...,1
20791,are we sure this guy is actually qualified for...,0


In [17]:
x = df["text"]
Y = df["class"]

In [18]:
#partitioning
x_train, x_test, y_train, y_test = train_test_split(x, Y, test_size = 0.25)

In [19]:
#text vectorization using TFID
from sklearn.feature_extraction.text import TfidfVectorizer

In [20]:
vectorization = TfidfVectorizer()
xv_train = vectorization.fit_transform(x_train)
xv_test = vectorization.transform(x_test)

## Classification

### Logistic Regression

In [21]:
from sklearn.linear_model import LogisticRegression

In [22]:
LR = LogisticRegression()
LR.fit(xv_train, y_train)

LogisticRegression()

In [23]:
LR.score(xv_test, y_test)

0.9872549019607844

In [24]:
pred_LR = LR.predict(xv_test)

In [25]:
print(classification_report(y_test, pred_LR))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      5985
           1       0.99      0.99      0.99      5235

    accuracy                           0.99     11220
   macro avg       0.99      0.99      0.99     11220
weighted avg       0.99      0.99      0.99     11220



 ### Decision Tree Classification

In [26]:
from sklearn.tree import DecisionTreeClassifier

In [27]:
DT = DecisionTreeClassifier()
DT.fit(xv_train, y_train)

DecisionTreeClassifier()

In [28]:
DT.score(xv_test, y_test)

0.9959893048128342

In [29]:
pred_DT = DT.predict(xv_test)

In [31]:
#print classification report
print(classification_report(y_test, pred_DT))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      5985
           1       1.00      1.00      1.00      5235

    accuracy                           1.00     11220
   macro avg       1.00      1.00      1.00     11220
weighted avg       1.00      1.00      1.00     11220



### Gradient Boosting Classifier

In [32]:
from sklearn.ensemble import GradientBoostingClassifier

In [34]:
GBC = GradientBoostingClassifier(random_state=0)
GBC.fit(xv_test, y_test)

GradientBoostingClassifier(random_state=0)

In [35]:
GBC.score(xv_test, y_test)

0.9982174688057041

In [36]:
pred_GBC = GBC.predict(xv_test)

In [37]:
print(classification_report(y_test, pred_GBC))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      5985
           1       1.00      1.00      1.00      5235

    accuracy                           1.00     11220
   macro avg       1.00      1.00      1.00     11220
weighted avg       1.00      1.00      1.00     11220



### Random Forest Classifier

In [38]:
from sklearn.ensemble import RandomForestClassifier

In [39]:
RFC = RandomForestClassifier(random_state=0)
RFC.fit(xv_train, y_train)

RandomForestClassifier(random_state=0)

In [40]:
RFC.score(xv_test, y_test)

0.9887700534759358

In [41]:
pred_RFC = RFC.predict(xv_test)

In [44]:
print(classification_report(y_test, pred_RFC))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      5985
           1       0.99      0.98      0.99      5235

    accuracy                           0.99     11220
   macro avg       0.99      0.99      0.99     11220
weighted avg       0.99      0.99      0.99     11220

